## Writeup for Advanced Lane Finding Project

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

My Jupyter notebook code is named Advanced Lane Finding.


### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

### Camera Calibration

Before working on a camera, a camera needs to be calibrated to remove any distortions
in the images captured by this camera. To undistort any image captured, we need to calibrate this camera.
In this project, we assume that we use a single camera through out.

To calibrate camera, I used opencv to obtain calibration matrix and distortion coefficient. So, I calibrated this camera by capturing multiple images of a known chessboard. Then, assuming the chessboard image is in (x,y) plane with z=0, I collect all object points for each image and then every time corners are detected on a chessboard 
image points are also collected. 

I used openCV "cv2.findChessboardCorners" to find corners. Corners were detected for most of the images
in folder "camera_cal" but failed to detect corners in three images.

Then by using openCV code "cv2.calibrateCamera", I calibrated the camera and obtained 
calibration matrix for this camera. I wrote a function which takes in image and object points to calibrate a camera. 
This calibration matrix is used through out the project code to undistort any images taken by this camera.

Function used to calibrate the camera is named,'calibrate_camera()', which takes in the image captured by camera and image and object points. It returns calibration matrix, and distortion coefficients to undistort any image captured by this calibrated camera. 

All chessboard images with corners detected and drawn can be found in folder,'chessboard_corners_image'.
Here is an example image displaying corners detected on a chessboard image.

![alt text](./chessboard_corner_images/corners_drawn13.jpg "Detected Corners")



Using the calibration matrix and distortion coefficient, a function named 'undistort_image()' is written to remove any distortion present in images. This function takes in distorted images, calibration matrix, and distortion coefficient to output undistorted images. 
Undistorted calibrated images can be found in folder 'undist_calibrated images' along with original images.
An example of an undistorted calibrated image is shown here. As can be seen undistorted image is a bit stretched indicating that some distortion is present along the borders of image captured by this camera. 

 
![alt text](./undist_calibrated_images/Original_image3.jpg "Original Image")
![alt text](./undist_calibrated_images/undist_calibrated_image3.jpg "Undistorted Calibrated Image")


### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.

Once the camera is calibrated, we can undistort any image captured by this camera. Distorted images from folder 'test_images' are all passed through the function 'undistort_image()' to remove any distortion present.
These undistorted test images can  be found in folder 'undist_test_images'.
An example image is shown here
![alt text](./undist_test_images/original_image1.jpg "Original Image")
![alt text](./undist_test_images/undist_image1.jpg "Undistorted Image")

#### 2. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

I'm performing perspective transform before color and gradient thresholding, because I think there is no point in applying color and gradient thresholding for the whole image, while we are wanting to find lane line pixels from a birds eye view image. So, in my jupyter notebook, you will see perspective transform being applied on undistorted image before passing it through color and gradient thresholding functions. 
I wrote a function 'perspective_transform()' to do perspective transform. It takes in an undistorted image and using source and destination points, a birds eye view warped image is obtained. I chose source and destination points using the width and height of undistorted image as I wanted them to be robust for any image passed in. I verified perspective transform by drawing source and destination points on original undistorted image. I checked if source points on original image are covering the area I'm interested in and that whether destination points are parallel or not.
All images from folder 'test_images' are passed through 'perspective_transform()' function and output images can be found in folder 'perspective_transform_test_images'.
Here is an example of original undistorted image with source and destination points along with birds eye view image.
![alt text](./perspective_transform_test_images/original_image0.jpg "Original Image")
![alt text](./perspective_transform_test_images/birds_view_image0.jpg "Birds-eye-view Image")

#### 3. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

For color and gradient thresholding, I initially ran few tests on color thresholding by varying threshold values for different color spaces like RGB and HLS on test images. After a couple of tests, I decided H and S channels serving best with proper threshold values in detecting white and yellow color lanes pixels. I wrote a function 'color_threshold()' which takes in warped image along with threshold values for R, H, L and S. Then it outputs corresponding binary thresholded images. 

Then, I wrote functions 'abs_sobel_thresh()', 'mag_thresh()', 'dir_thresh()' to compute absolute gradients, magnitude of gradients, and direction of gradients respectively within provided threshold values. Each function takes in warped image and threshold values, while outputting a corresponding binary image meeting threshold values. 

Then, function 'gradient_thresh()' is used to experiment with various threshold values combining results of mag_thresh and dir_thresh outputs. 

Finally, function 'color_grad_threshold()' is used to combine both color and gradient thresholdings to get best of both.

Color thresholded binary images can be found in folder 'color_threshold_images'. 
Below is an example binary image output. As can be seen, for chosen threshold values white and yellow line pixels are detected pretty well. But, based on the provided image, chosen threshold values may or may not work in detecting white and yellow line pixels. Cases where tweeking threshold values is needed are, when the day is sunny, cloudy, trees shadowing the road, roads painted with colors other than white and yellow e.t.c


![alt text](./color_threshold_images/original_image0.jpg "Original Image")
![alt text](./color_threshold_images/color_threshold_image0.jpg "Color Thresholded Image")


#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

Once the yellow and white color pixels are detected, we need to find pixels which represent the actual lanes. For this, 'hist()' function is used to find the start point along x-direction. The binary image consisting of 0s and 1s is used and summed values columnwise to find the peak value. Peak value will be found along width where more pixels are populated vertically. So the two most prominent peaks in this histogram will be good indicators of the x-position of the base of the lane lines. We can use that as a starting point for where to search for the lines. From that point, we can use a sliding window, placed around the line centers, to find and follow the lines up to the top of the frame. Inside 'hist()' function, I'm summing up pixel values close to vehicle as when road is very curvy, these are the pixels which are most likely vertical.  

After trial and error, I have chosen hyperparameters for sliding window approach as in function 'find_lane_pixels()'. This is the function which uses sliding window approach in detecting lane pixels. Using polyfit, I fitted a second order polynomila for all detected lane pixels. 
All images with detected lane pixels can be found in folder 'detected_lane_pixels_images'. 

Below is an example image where lane pixels are detected and fitted a polynomial on those pixels using sliding window approach.

![alt text](./detected_lane_pixels_images/warped_original_image0.jpg "Original Image")
![alt text](./detected_lane_pixels_images/detected_lane_pixels_image0.jpg "Detected Lane Lines Image")
![alt text](./detected_lane_pixels_images/sliding_window_image.jpg "Sliding Window Image")


#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

Once the lane pixels are found and a polynomial is fitted, we have the lines representing the lanes. Using the equation 
$R_{curve}$ = $\frac{[1 + (\frac{dx}{dy})^2]^{1.5}}{|{\frac{d^2x}{dy^2}}|}$, radius of curvature is computed for each lane line detected. 
After comparing images with U.S. regulations that require a minimum lane width of 12 feet or 3.7 meters, and the dashed lane lines are 10 feet or 3 meters long each, below conversion factors of meters to pixels along x and y directions are chosen. 

ym_per_pix = 30/720
xm_per_pix = 3.7/700

These two radii of curvatures of both lane lines is averaged and is displayed on the image. 
Then, camera center is computed by taking the midpoint of left and right lane starting points. Then offset is computed by comparing the difference in positions of camera center and midpoint of undistorted image width. If the offset is positive, it implies that the vehicle is positioned offset distance from camera center. 
Offset is also displayed on the output image. 


#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

Pipeline named 'advanced_lanes_pipeline()', takes in an image from a calibrated camera, undistorts the image and obtains birds-eye-view perspective transfromed image for color and gradient thresholding to detect yellow and white lane line pixels. These detected pixels are fitted onto a second order polynomial. Once the polynomials are obtained, the radius of curvature is computed and offset of the vehicle position relative to camera center are displayed onto original image along with the lane area. 

An example image passed through the pipeline is as shown below. 
All test images are passed throught the pipeline and resulting output images can be found in folder 'pipeline_images'.

![alt text](./pipeline_images/pipeline_original_image5.jpg "Original Image")
![alt text](./pipeline_images/pipeline_image5.jpg "Pipeline Image")

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

All three videos, project_video, challenge_video, harder_challenge_video are passed through the pipeline 'advanced_lanes_pipeline()'.
Output videos can be found in folder 'test_videos'.
Project_video seems good but challenge and harder_challenge videos are not as I expected.


### Discussion


#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

I encountered problems in color and gradient thresholding. Color and gradient thresholding is very important as everything else in the project relies on it. If yellow and white color pixels pertaining to lane lines are not detected correctly, then we might fit wrong polynomial and obtain a narrow or wide lane. This is a huge problem!
So, I had to struggle with various threshold values for colors spaces rgb, hls and gradients along x and y directions. After various trial and errors, I chose threshold values which are reasonably working fine. But, color and gradient threshold values I chose maight not work for few cases when road is covered with snow, or shadows or if it is painted with different colors. For which I believe we could make use of various color spaces like HLS, HSV, LAB, YCrCb to detect yellow and white line pixels accurately under different situations.  

Also, perspective transform has to be more dynamic in cases of steep and curvy roads.  
Then, based on the detected pixels, I obtain starting points for left and right lanes to be drawn. We use histogram to obtain starting points to fit polynomials for detected lane pixels using bottom half of image assuming these pixels are vertical. But for curvy roads we need to choose lane pixels that are very close to vehicle as these are the pixels which are most likely to be  vertical. So, for curvy roads we may need to use pixels values that are vertical and close to the vehicle. 

Pipeline worked fine for project_video and reasonably okay for challenge_video but it seemed to fail for harder_challenge_video with very curvy lanes. For this video, pipeline failed to draw lines because the lanes are very curvy and the road seemed steep. Becasue of this perspective transform failed to get correct birds-eye-view. Apart from that few camera frames didn't capture the lane lines at all because of which lane line pixels and lanes were not appropriatly drawn. If source and destination points for perspective transform are made more dynamic, I think this issue could be avoided. and, for instances where no lane lines are captured by camera, using the detected line we can draw a virtual lane line knowing the width between two lanes and radius of curvature of detected lane line.

I planned to calibrate my own camera, undistort images captured by my camera and draw lane lines for a video taken from that camera. 
I succesfully calibrated the camera, and undistorted the images taken by camera. These images can be found in folders, 'chessboard_corner_images2' and 'undist_test_images2'. 

For remaining part of lane line pixels detection and drawing lane lines, I'm planning to finish it by end of this term. 

Doing so, I understood how challenging it is in implementing these concepts in real time. It gave me a flavour of how hard we need to work to have accurate our concepts and codes are need to be.


 